<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/llama_datasets/uploading_llama_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 将LlamaDataset 贡献给 LlamaHub


`LlamaDataset`的存储是通过一个git仓库进行管理的。要贡献一个数据集，需要向`llama_index/llama_datasets` Github（LFS）仓库发起一个拉取请求。

要贡献一个`LabelledRagDataset`（`BaseLlamaDataset`的子类），需要两组文件：

1. 以`rag_dataset.json`命名的json格式的`LabelledRagDataset`文件
2. 用于创建`LabelledRagDataset`的源文档文件

这个简短的笔记本提供了一个使用Paul Graham Essay文本文件的快速示例。


In [ ]:
%pip install llama-index-llms-openai

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### 加载数据


In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

In [ ]:
from llama_index.core import SimpleDirectoryReader

# 加载文档并构建索引
documents = SimpleDirectoryReader(
    input_files=["data/paul_graham/paul_graham_essay.txt"]
).load_data()

In [ ]:
# 生成与块相对应的问题
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.llms.openai import OpenAI

# 为llm提供者设置上下文
llm_gpt35 = OpenAI(model="gpt-4", temperature=0.3)

# 实例化一个DatasetGenerator
dataset_generator = RagDatasetGenerator.from_documents(
    documents,
    llm=llm_gpt35,
    num_questions_per_chunk=2,  # 设置每个节点的问题数量
    show_progress=True,
)

rag_dataset = dataset_generator.generate_dataset_from_nodes()

现在我们已经生成了我们的`LabelledRagDataset`（顺便说一句，完全可以手动创建一个，使用人工生成的查询和参考答案！），我们将其存储到必要的json文件中。


In [ ]:
rag_dataset.save_json("rag_dataset.json")

#### 生成基准结果

除了添加一个`LlamaDataset`之外，我们还鼓励添加基准基准测试，供其他人用作衡量他们自己的RAG管道的参照。


In [ ]:
from llama_index.core import VectorStoreIndex

# 一个基本的RAG管道，使用默认设置
index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()

# 手动
prediction_dataset = await rag_dataset.amake_predictions_with(
    query_engine=query_engine, show_progress=True
)

## 提交拉取请求



使用`rag_dataset.json`和源文件`paul_graham_essay.txt`（请注意，在这种情况下，只有一个源文件，但也可以有多个），我们可以执行两个步骤来将`LlamaDataset`贡献到`LlamaHub`中：

1. 类似于为`loader`、`agent`和`pack`做出贡献的方式，为`llama_hub`存储库创建一个拉取请求，添加一个新的文件夹用于新的`LlamaDataset`。这一步将上传有关新`LlamaDataset`的信息，以便在`LlamaHub`用户界面中呈现。

2. 创建一个拉取请求到`llama_datasets`存储库，实际上传数据文件。


### 步骤 0（先决条件）

首先，fork并克隆（到本地计算机）`llama_hub` Github仓库和`llama_datasets`仓库。您将从fork版本的新分支向这两个仓库提交pull请求。


### 第1步

在`llama_hub` Github仓库的`llama_datasets/`中创建一个新文件夹。例如，在这种情况下，我们将创建一个名为`llama_datasets/paul_graham_essay`的新文件夹。

在该文件夹中，需要两个文件：
- `card.json`
- `README.md`

特别是在您的本地机器上：

```
cd llama_datasets/
mkdir paul_graham_essay
touch card.json
touch README.md
```

建议在此处查看先前提交的`LlamaDataset`，根据需要修改其相应文件以适应您的新数据集。


在我们当前的示例中，我们需要`card.json`看起来如下所示

```json
{
    "name": "Paul Graham Essay",
    "description": "一个基于Paul Graham的一篇文章的标记为RAG的数据集，包括查询、参考答案和参考上下文。",
    "numberObservations": 44,
    "containsExamplesByHumans": false,
    "containsExamplesByAI": true,
    "sourceUrls": [
        "http://www.paulgraham.com/articles.html"
    ],
    "baselines": [
        {
            "name": "llamaindex",
            "config": {
                "chunkSize": 1024,
                "llm": "gpt-3.5-turbo",
                "similarityTopK": 2,
                "embedModel": "text-embedding-ada-002"
            },
            "metrics": {
                "contextSimilarity": 0.934,
                "correctness": 4.239,
                "faithfulness": 0.977,
                "relevancy": 0.977
            },
            "codeUrl": "https://github.com/run-llama/llama_datasets/blob/main/baselines/paul_graham_essay/llamaindex_baseline.py"
        }
    ]
}
```


对于`README.md`，这些都是非常标准的，要求您在`download_llama_dataset()`函数调用中更改数据集参数的名称。

```python
from llama_index.llama_datasets import download_llama_datasets
from llama_index.llama_pack import download_llama_pack
from llama_index import VectorStoreIndex

# 下载并安装rag评估器包的依赖项
RagEvaluatorPack = download_llama_pack(
  "RagEvaluatorPack", "./rag_evaluator_pack"
)
rag_evaluator_pack = RagEvaluatorPack()

# 下载并安装基准数据集的依赖项
rag_dataset, documents = download_llama_datasets(
  "PaulGrahamEssayTruncatedDataset", "./data"
)

# 评估
query_engine = VectorStoreIndex.as_query_engine()  # 先前定义，此处未显示
rag_evaluate_pack.run(dataset=paul_graham_qa_data, query_engine=query_engine)
```


最后，第一步的最后一项是创建一个条目到`llama_datasets/library.json`文件中。在这种情况下：

```json
    ...,
    "PaulGrahamEssayDataset": {
    "id": "llama_datasets/paul_graham_essay",
    "author": "andrei-fajardo",
    "keywords": ["rag"],
    "extra_files": ["paul_graham_essay.txt"]
  }
```

注意：`extra_files`字段是用于存储源文件的。


### 第2步 上传实际数据集

在这一步中，由于我们在`llama_datasets`仓库中使用了Github LFS，因此进行贡献的方式与我们的其他开放Github仓库的贡献方式完全相同。也就是说，提交一个拉取请求。

首先，fork `llama_datasets`仓库，并在`llama_datasets/`目录中创建一个新文件夹，该文件夹与在`library.json`文件中创建的条目的`id`字段相匹配。因此，对于这个示例，我们将创建一个名为`llama_datasets/paul_graham_essay/`的新文件夹。我们将在这里添加文档并创建拉取请求。

在这个文件夹中，添加`rag_dataset.json`（必须命名为这个），以及其余的源文档，对于我们的情况来说是`paul_graham_essay.txt`文件。

```sh
llama_datasets/paul_graham_essay/
├── paul_graham_essay.txt
└── rag_dataset.json
```

现在，只需`git add`，`git commit`和`git push`您的分支，然后创建您的PR。
